In [1]:
import cenpy
import osmnx
import contextily
import networkx as nx
import pandas as pd
import pickle
import geopandas
import numpy as np

In [2]:
def return_layers(place):
    '''INPUT: string query for place of interest
       OUTPUT: list of networkx graphs, each representing a different road type layer'''
    G_1 = osmnx.graph_from_place(place,retain_all=True,
                          custom_filter='["highway"~"cycleway|path|living_street"]["bicycle"!~"no"]')
    G_2 = osmnx.graph_from_place(place,retain_all=True,custom_filter='["cycleway"~"lane"]')
    L1 = nx.compose(G_1,G_2)

    G_1 = osmnx.graph_from_place(place,network_type='all',simplify=True,retain_all=True,
                                   custom_filter='["highway"~"cycleway|path|living_street|residential"]["bicycle"!~"no"]')
    G_2 = osmnx.graph_from_place(place,retain_all=True,custom_filter='["cycleway"~"lane"]')
    L2 = nx.compose(G_1,G_2)

    L3 = osmnx.graph_from_place(place,retain_all=True,network_type='bike',simplify=True)
    L4 = osmnx.graph_from_place(place,retain_all=True,network_type='drive',simplify=True) 
    return [L1,L2,L3,L4]

In [3]:
def get_nodes(graph,dem_data):
    '''INPUT: networkx graph representing a single road type layer
       OUTPUT: GeoDataFrame of nodes in the graph, with adjusted geometry'''
    # convert networkx data to GeoDataFrame form
    nodes, streets = osmnx.graph_to_gdfs(graph)
    
    # make the coordinate reference systems of ACS data and OSM data the same (Web Mercator)
    nodes = nodes.to_crs(dem_data.crs)
    # update coordinate columns based on new crs geometry
    nodes['x'] = nodes['geometry'].apply(lambda g: g.coords[0][0])
    nodes['y'] = nodes['geometry'].apply(lambda g: g.coords[0][-1])
    return nodes

In [4]:
def get_streets(graph,dem_data):
    '''INPUT: networkx graph representing a single road type layer
       OUTPUT: GeoDataFrame of streets/edges in the graph, with adjusted geometry'''
    # convert networkx data to GeoDataFrame form
    nodes, streets = osmnx.graph_to_gdfs(graph)
    
    streets = streets.to_crs(dem_data.crs)
    return streets

In [29]:
def get_data(place,level='tract',variables=None):
    '''INPUT: string query for place of interest
       OUTPUT: DataFrame of demographic data, 
               list of GeoDataFrames corresponding to nodes from different road type layers,
               list of GeoDataFrames corresponding to edges from different road type layers'''
    # pull demographic data from ACS
    acs = cenpy.products.ACS()
    dem_data = acs.from_place(place,place_type="Incorporated Place",level=level,variables=variables)
    # pull street network data from OSM
    graphs = return_layers(place)
    nodes = [get_nodes(graph,dem_data) for graph in graphs]
    streets = [get_streets(graph,dem_data) for graph in graphs]
    
    return dem_data,nodes,streets

In [33]:
place = 'Detroit,MI'
dem,nodes,streets = get_data(place)

Matched: Detroit,MI to Detroit city within layer Incorporated Places


C:\Users\kyan3\AppData\Local\Temp\ipykernel_14608\3924244956.py:8: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  dem_data = acs.from_place(place,place_type="Incorporated Place",level=level,variables=variables)


In [34]:
dem.to_file('data/Detroit_dem')
for ix,node_list in enumerate(nodes):
    node_list.to_file('data/Detroit_nodes_'+str(ix))
for ix,street_list in enumerate(streets):
    street_list = street_list[['highway','length','geometry']]
    # convert any lists to strings in highway column
    street_list['highway'] = [''.join(h) for h in street_list['highway']]
    street_list.to_file('data/Detroit_streets_'+str(ix))

C:\Users\kyan3\AppData\Local\Temp\ipykernel_14608\3991246571.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  node_list.to_file('data/Detroit_nodes_'+str(ix))
C:\Users\kyan3\AppData\Local\Temp\ipykernel_14608\3991246571.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  node_list.to_file('data/Detroit_nodes_'+str(ix))
C:\Users\kyan3\AppData\Local\Temp\ipykernel_14608\3991246571.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  node_list.to_file('data/Detroit_nodes_'+str(ix))
C:\Users\kyan3\AppData\Local\Temp\ipykernel_14608\3991246571.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  node_list.to_file('data/Detroit_nodes_'+str(ix))
C:\Users\kyan3\miniconda3\envs\ox\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying t